Versions:
* v1: one full cycle. Not yet multiple cycles implemented
* v2: MORDM cycle into IO-function
    * Can be used for MORDM validation
* v3: restructure into MORDM function
* v4: mpmordm iterative way

TODO inputdata: 
* FL not random generation
* distances based on OSRM route matrix

TODO model:
* Check values for disruption 1-2 of 0-1

TODO:
* #Filter out those where no new demand is covered<br>
    #TODO: Works only first time. After that doesn't work.<br>
    #TODO: Change to "not more than previous round"
* XX done via saving: return information about objective prioritisation for each returned nondom policy and nondom robust policy
* better names inputvariables
* change info based on locations

# Concept

Recursion

In [1]:
def ideaMordm(options, split_into = 2, max_depth = 2):
    for option in options:
        if len(option) == max_depth:
            print("max depth reached", option)
        else:
            for n in range(split_into):
                new_option = [option + [n], option + [n]]
                print(new_option)
                ideaMordm(new_option,split_into,max_depth)
    global a
    a = 2

In [2]:
ideaMordm([[0]])

[[0, 0], [0, 0]]
max depth reached [0, 0]
max depth reached [0, 0]
[[0, 1], [0, 1]]
max depth reached [0, 1]
max depth reached [0, 1]


Iterative (better)

In [3]:
periods = 3
nodes = {"p{}".format(p): [] if p !=0 else ["n{}".format(p)] for p in range(periods)}

node_counter = 0

for p in range(periods):
    for ni, n in enumerate(nodes["p{}".format(p)]):
        print(p,n)
        if p+2 <= periods:
            # in func
            node_counter +=1
            nodes["p{}".format(p+1)].append(n+"p{}n{}".format(p,node_counter))
            #
            node_counter +=1
            nodes["p{}".format(p+1)].append(n+"n{}".format(node_counter))

    print("__end_period:",p)

0 n0
__end_period: 0
1 n0p0n1
1 n0n2
__end_period: 1
2 n0p0n1p1n3
2 n0p0n1n4
2 n0n2p1n5
2 n0n2n6
__end_period: 2


In [108]:
all_nodes = []

In [110]:
for p in range(periods):
    all_nodes.append(nodes["p{}".format(p)])

In [112]:
for p in all_nodes:
    print(len(p))

1
2
4
8
16
32
64
128
256
512


# Imports

In [4]:
# Standard imports & printing versions
import random
import pandas as pd
import numpy as np

# Own Model import
from lib.fl_model_v5 import *
# import lib.fl_model_v5 as flm

# for natural sorting
import re

#for nondominated sorting
import lib.pareto as pareto
from lib.list_imports import find_loaded_modules

# for parallel plotting
from lib.parallel_plotting import plot_optimal

# For checking ema running time
import time

import ema_workbench as ema
from ema_workbench.em_framework import (Policy, IntegerParameter, Constant,
                                        RealParameter, ScalarOutcome,
                                        perform_experiments, Model)
from ema_workbench import ema_logging
# from ema_workbench.em_framework.

ema_logging.log_to_stderr(ema_logging.INFO)

import sys

In [5]:
# %load https://gist.github.com/TRomijn/d9d28ba7b7d5eeac1deea5a26dd326b2/raw/loaded_modules.py

'''
original author: kinverarity1

List loaded modules and packages, and show their version numbers
and/or Git repository's HEAD commit SHA.


Changes:
Minor changes to make compatible with Python 3
'''
# Standard library modules
import types
import os

# Third-party packages
import git      # GitPython


def module_path(mod):
    '''Returns path to the file that module *mod* comes from.
    If it doesn't come from a file, return None.'''
    if hasattr(mod, '__file__'):
        return os.path.abspath(os.path.dirname(mod.__file__))
    else:
        return None

    
def from_git_repo(mod):
    '''Does the module *mod* reside in a Git repository?'''
    path = module_path(mod)
    if path:
        try:
            repo = git.Repo(path)
        except:
            return False
        else:
            return True
    else:
        return False

    
def git_path_sha(mod, slice=slice(0, 8, 1)):
    '''Return SHA hash for the HEAD commit for the repository
    that the module *mod* resides in.'''
    repo = git.Repo(module_path(mod))
    return repo.git_dir, repo.head.commit.hexsha[:8]


def module_version(mod):
    '''Return version string for module *mod*, or nothing if
    it doesn't have a "version" or "__version__" attribute.'''
    version = []
    if hasattr(mod, '__dict__'):
        keys = []
        for key in mod.__dict__.keys():
            if key.lower() == 'version' or key.lower() == '__version__':
                v = mod.__dict__[key]
                if isinstance(v, str):
                    version.append(v)
        if keys:
            print (mod, keys)
    if version:
        return ', '.join(version)
    else:
        return ''

    
def find_loaded_modules(only_versioned_modules=True):
    '''Return list of loaded modules for which there is a version
    number or a Git repository commit SHA.
    
    Return a list of *(name, version, path_to_git_repo, git_head_sha)*,
    which has an HTML property for pretty display in IPython Notebooks.
        
    '''
    def list_of_lists_to_HTML(lists, header_row=None):
        '''Convert a list of a list of strings to a HTML table.'''
        s = '<table>'
        if header_row:
            s += '\n\t<tr>\n\t\t'
            s += ''.join(['<th>%s</th>' % item for item in header_row])
            s += '\n\t</tr>'
        for inner_list in lists:
            s += '\n\t<tr>\n\t\t'
            s += ''.join(['<td>%s</td>' % item for item in inner_list])
            s += '\n\t</tr>'
        s += '\n</table>'
        return s
    
    class LoadedModules(list):
        '''Very simple wrapper for a list of lists of strings, with an attribute
        for display in IPython Notebooks.'''
        def __init__(self, *args, **kwargs):
            list.__init__(self, *args, **kwargs)
            
        @property
        def HTML(self):
            from IPython.display import HTML
            return HTML(
                    list_of_lists_to_HTML(
                            self, header_row=['Name', 'Version', 'Path', 'SHA']))
                    
    objs = LoadedModules()
    for i, mod in enumerate(globals().values()):
        if isinstance(mod, types.ModuleType):
            if hasattr(mod, '__name__'):
                name = mod.__name__
            else:
                name = ''
            
            if from_git_repo(mod):
                path, sha = git_path_sha(mod)
            else:
                path = ''
                sha = ''
            
            version = module_version(mod)
            
            if only_versioned_modules:
                flag = version or (path and sha)
            else:
                flag = True
            
            if flag:
                objs.append([mod.__name__, version, path, sha])
    objs.sort(key=lambda r: r[0])
    return objs


    
    

In [6]:
find_loaded_modules().HTML

Name,Version,Path,SHA
ema_workbench,1.1.3,,
git,2.1.9,,
json,2.0.9,,
lib.pareto,1.1.1-3,,
numpy,1.14.2,,
pandas,0.22.0,,
re,2.2.1,,
sys,"3.6.4 |Anaconda, Inc.| (default, Mar 12 2018, 20:20:50) [MSC v.1900 64 bit (AMD64)]",,


# Data Prep

## Load Data

Data is preliminary data. TODO: Find reliable input data

In [7]:
# Create DPs from Nepal Data
DPs = pd.read_csv("Data/Nepal Cities Population.csv", usecols=[1,2,3,4])

# Take only larger cities, because of many null values under 50000 inhabitants. 
# TODO find better dataset with population and coordinates
DPs = DPs [DPs.Population >= 50000]
DPs.head(2)

,City,Population,Latitude (DD),Longitude (DD)
0,Kathmandu,743300,27.71,85.31
1,Biratnagar,178000,26.46,87.28


In [8]:
# Create SPs from Nepal Data
airports = pd.read_csv("Data/Nepal Airports.csv", usecols=[0,5,6])
SPs = airports[airports['Airport'] == "Tribhuvan intl"]
SPs.head(2)

,Airport,Latitude (DD),Longitude (DD)
7,Tribhuvan intl,27.7,85.36


In [9]:
# Create random FLs in Nepal
long = np.random.uniform(DPs['Longitude (DD)'].min(),
                         DPs['Longitude (DD)'].max(), 20)
lat = np.random.uniform(DPs['Latitude (DD)'].min(), DPs['Latitude (DD)'].max(),
                        20)
FLs = pd.DataFrame([lat, long], index=['Lat', 'Lon']).T
FLs['Name'] = ['FLO{}'.format(i) for i in range(FLs.shape[0])]
FLs = FLs[['Name', 'Lat', 'Lon']]
FLs.head(2)

,Name,Lat,Lon
0,FLO0,28.058248,84.283301
1,FLO1,27.236077,80.858178


## Prepare Data for model input

### Global: Fixed certain data

In [10]:
# Coordinates
# all large cities to demand points
DPY = {"DPY{}".format(i): y for i,y in enumerate(DPs['Latitude (DD)'])}
DPX = {"DPX{}".format(i): x for i,x in enumerate(DPs['Longitude (DD)'])}

# all international airports to supply points:
SPY = {"SPY{}".format(i): y for i,y in enumerate(SPs['Latitude (DD)'])}
SPX = {"SPX{}".format(i): x for i,x in enumerate(SPs['Longitude (DD)'])}

# Facility locations
FLX = {"FLX{}".format(i): x for i,x in enumerate(FLs['Lon'])}
FLY = {"FLY{}".format(i): y for i,y in enumerate(FLs['Lat'])}

# Population demand points
DPpop = {"DPpop{}".format(i): pop for i,pop in enumerate(DPs['Population'])}
# DPpop = {"DPpop{}".format(i): random.uniform(10,100) for i in range(nr_of_DPs)}


### Global: Fixed uncertain data

In [11]:
global_fixed_unc = pd.read_csv("Data/uncertainties/fixed_uncertainties.csv").set_index('var')

### Variable Uncertain Data: generate random for now:
DFs becomes a global. can be accessed by using its label

In [12]:
# DSR = Disruption
DSRDP = {"DSRDP{}".format(i): random.uniform (1, 2) for i in range(len(DPX))}
DSRFL = {"DSRFL{}".format(i): random.uniform (1, 2) for i in range(len(FLX))}


In [13]:
# label each branch. start with:
start_label = "p0n0"
# Each branch has a separate perception of what the data is.
# A dictionary can keep track of data for each branch, linked via label
global_dfs_var_unc_data = {}

In [14]:
global_dfs_var_unc_data [start_label] = pd.DataFrame.from_dict({**DSRDP, **DSRFL}, orient='index')

In [15]:
global_dfs_var_unc_data [start_label].head(1)

,0
DSRDP0,1.363024


In [16]:
# create upper and lower bound
global_dfs_var_unc_data[start_label] = global_dfs_var_unc_data[
    start_label].rename(columns={0: "actual"})
global_dfs_var_unc_data[start_label]['lower'] = (
    global_dfs_var_unc_data[start_label]['actual'] - 1
) * np.random.uniform(size=global_dfs_var_unc_data[start_label].shape[0]) + 1
global_dfs_var_unc_data[start_label]['upper'] = (
    2 - global_dfs_var_unc_data[start_label]['actual']) * np.random.uniform(
        size=global_dfs_var_unc_data[start_label]
        .shape[0]) + global_dfs_var_unc_data[start_label]['actual']
global_dfs_var_unc_data[start_label]['best_estimate'] = (
    global_dfs_var_unc_data[start_label]['upper'] +
    global_dfs_var_unc_data[start_label]['lower']) / 2
global_dfs_var_unc_data[start_label][
    'difference'] = global_dfs_var_unc_data[start_label]['best_estimate'] - global_dfs_var_unc_data[start_label]['actual']
global_dfs_var_unc_data[start_label].head(1)

,actual,lower,upper,best_estimate,difference
DSRDP0,1.363024,1.138707,1.699101,1.418904,0.05588


In [17]:
global_dfs_var_unc_data.keys()

dict_keys(['p0n0'])

### Other Globals:
- Create Starting situation policies:
- List of all Models

In [18]:
# global_policy_tree = {}
# # global_models = {}
# # global_period_counter = 0 # period is not global
# global_node_counter = 0
# EMA_model_dict = {}  # XX somewhere else: global
# print("starting label:",start_label)

# MORDM

## Helper functions

In [19]:
def create_policies(FL_dict, print_pols=False):
    """
    Takes the current network of Facility Locations and returns all possible policies for one added FL.
    
    Input: current option (Dict of FLs)
    Output: List of options (Dicts of FLs)
    
    Printing module not completely reliable. doesnt print whats actually in thereTODO
    """
    def natural_key(string_):
        return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_)]
    
    # All FL names
    FL_keys = sorted(FL_dict.keys(),key=natural_key)
    # All FL values indicating whether operational
    FL_values = [FL_dict[key] for key in FL_keys]
    
    i_operational_fls = i_FL_op = [i for i,x in enumerate(FL_values) if x == 1]
    
    # Create list of policies  
    pols = np.identity(len(FL_keys))
    # Set already operational facilities to operational
    pols[:,i_operational_fls] = 1
    
    #Delete policies where no new FLs are placed 
    new_n_operational_fls = sum(FL_values) + 1
    pols = pols[pols.sum(axis = 1) == new_n_operational_fls]

    
    # Return a list of dictionaries
    policy_list = []
    for pol in pols:
        policy_list.append({key:value for key,value in zip(FL_keys,pol)})
    
    if print_pols == True:
        print("total policies:",len(policy_list))
        for n,i in enumerate(policy_list):
            for v in i.values():
                print (int(v), end='')
            print(" <- policy {}".format(n))
                
    return policy_list

In [20]:
def calculate_signalNoise(data, obj):
    '''
    data : 1d array, the values of an outcome indicator for a policy
    obj  : the objective corresponding to an outcome indicator, 1 or -1
    '''
    if obj == -1: #MINIMIZE
        score = (np.mean(data)+1)*(np.std(data)+1) #to avoid division by zero if the std. deviation is zero, we can add 1.
    elif obj == 1: #MAXIMIZE
        score = (np.mean(data)+1)/(np.std(data)+1)
    return score

In [21]:
def calculate_maxregret(data, obj):
    '''
    data : a list of 1d arrays whose shape is no_policies x no_scenarios
    obj : -1 or 1, the objective 
    '''
    data = np.array(data)
    if obj == 1: #MAXIMIZE 
        #find the best case in each scenario, therefore use the max function of numpy on the axis of policies
        zero_regrets = np.max(data, axis=0)
    elif obj == -1: #MINIMIZE
        zero_regrets = np.min(data, axis=0)
    
    #determine the regret values for eaxh policy in each scenario
    regrets = [abs(np.subtract(data[p], zero_regrets)) for p in range(data.shape[0])]
    
    max_regrets = np.max(regrets, axis=1)
    
    return max_regrets

## Main

In [22]:
def MORDM(
        current=None,
        current_period=None,
        trace_label=None,
        # fixed_data=None,
        # var_data=None,
        n_scenarios=100,
        additional_info=None,
        show_MORDM_graphics=False,
        show_model_graphics=False):
    """
    current: indexes of current operational locations
    trace_label: label of input node
    fixed_data: 
    var_data: df of variable data for each 
    max_depth: maximum number of total FLS
    """
    #     if sum(current.values()) >= max_depth:
    #         raise NotImplementedError(
    #             "Everything might go right(no error). This function is not well implemented. Change to > instead of >="
    #         )

    if show_MORDM_graphics == True:
        raise NotImplementedError("""visualisations are not implemented yet. 
            Not sure yet if impementing graphics in MPMORDM is usefull for every period """
                                  )

    #Start Many Objective Optimisation
    #best estimate values
    best_estimate_disruptions = {
        i: be
        for be, i in zip(global_dfs_var_unc_data[trace_label]['best_estimate'],
                         global_dfs_var_unc_data[trace_label].index)
    }

    optional_policies = create_policies(current, print_pols=False)

    #evaluate all policies
    best_est_results_opt_pols = []
    for policy in optional_policies:
        best_est_results_opt_pols.append(
            FL_model(
                unit_opening_costs=global_fixed_unc.loc["unit_opening_costs",
                                                        "best_estimate"],
                unit_transport_cost=global_fixed_unc.loc["unit_transport_cost",
                                                         "best_estimate"],
                FL_operations_cost=global_fixed_unc.loc["FL_operations_cost",
                                                        "best_estimate"],
                graphical_representation=show_model_graphics,
                FL_range=global_fixed_unc.loc["FL_range",
                                              "best_estimate"],  # km? --> m
                dist_method="great_circle",
                lorry_speed=global_fixed_unc.loc[
                    "lorry_speed", "best_estimate"],  #km/u? --> m/s
                # fixed certain
                **SPX,
                **SPY,
                **DPX,
                **DPY,
                **FLX,
                **FLY,
                **DPpop,
                # specific
                **policy,
                **best_estimate_disruptions))

    # outcome indicators, as internally used by the model
    oois = [
        'total_costs', 'nr_uncovered_DPs', 'total_uncovered_demand',
        'max_distr_time'
    ]
    # Put outcomes in DF
    df_best_est_results_opt_pols = pd.DataFrame(
        np.asarray(best_est_results_opt_pols)[:, :4], columns=oois)

    #nondominated sorting
    nondominated_BE_policy_results = np.array(
        pareto.eps_sort(
            [list(df_best_est_results_opt_pols.itertuples(index=True))],
            [1, 2, 3, 4], [1e-9, 1e-9, 1e-9, 1e-9]))

    #Filter out those where new demand is covered
    #TODO: Works only first time. After that doesn't work.
    #TODO: Change to "not more than previous round"
    nondominated_BE_policy_results = nondominated_BE_policy_results[
        nondominated_BE_policy_results[:, 4] != 0]

    #put nondominated policies in array
    nondom_i = [int(a[0]) for a in nondominated_BE_policy_results]
    nondominated_BE_policies = np.array(optional_policies)[nondom_i]

    df_nondom_BE_pols = pd.DataFrame(
        nondominated_BE_policy_results, columns=["i"] + oois).set_index("i")
    df_nondom_BE_pols.to_csv(
        "results/mpmordm/nondom_BE_pols{}.csv".format(trace_label))


    #End Many Objective Optimisation
    #Start Robustness analysis

    EMA_model_dict[trace_label] = Model("flmodel{}".format(trace_label),
                                        FL_model)

    EMA_model_dict[trace_label].locations = [
        Constant("DPX{}".format(i), x)
        for i, x in zip(DPs.index, DPs['Longitude (DD)'])
    ] + [
        Constant("DPY{}".format(i), y)
        for i, y in zip(DPs.index, DPs['Latitude (DD)'])
    ] + [
        Constant("SPX{}".format(i), x)
        for i, x in zip(SPs.index, SPs['Longitude (DD)'])
    ] + [
        Constant("SPY{}".format(i), y)
        for i, y in zip(SPs.index, SPs['Latitude (DD)'])
    ] + [
        Constant("FLX{}".format(i), x) for i, x in zip(FLs.index, FLs['Lon'])
    ] + [
        Constant("FLY{}".format(i), y) for i, y in zip(FLs.index, FLs['Lat'])
    ]

    # model.locations_uncertain =   [
    #     RealParameter("FLX{}".format(i), DPs['Longitude (DD)'].min(), DPs['Longitude (DD)'].max()) for i in range(nr_of_FLs)
    # ] + [RealParameter("FLY{}".format(i), DPs['Latitude (DD)'].min(), DPs['Latitude (DD)'].max()) for i in range(nr_of_FLs)]

    EMA_model_dict[trace_label].constants = [
        Constant('graphical_representation', False),
        Constant('dist_method', 'great_circle'),
        Constant('Error_Test', 1),  # this doesn't do anything. 
        #list of other constants
        #     Constant
    ] + EMA_model_dict[trace_label].locations + [  # population DPs
        Constant("DPpop{}".format(i), pop)
        for i, pop in zip(DPs.index, DPs['Population'])
    ]

    # Reachability of Demand Points and Facility Locations can be disrupted
    # Disruption also determines demand (disr-1)*pop
    EMA_model_dict[trace_label].disruptions = [
        RealParameter(i, l, u) for i, l, u in zip(global_dfs_var_unc_data[
            trace_label].index, global_dfs_var_unc_data[trace_label][
                'lower'], global_dfs_var_unc_data[trace_label]['upper'])
    ]

    EMA_model_dict[trace_label].uncertainties = [
        RealParameter(i, l, u)
        for i, l, u in zip(global_fixed_unc.index, global_fixed_unc.lower,
                           global_fixed_unc.upper)
    ] + EMA_model_dict[trace_label].disruptions  #+ model.locations_uncertain

    EMA_model_dict[trace_label].outcomes = [
        ScalarOutcome("total_costs", kind=ScalarOutcome.MINIMIZE),
        ScalarOutcome("nr_uncovered_DPs", kind=ScalarOutcome.MINIMIZE),
        ScalarOutcome("total_uncovered_demand", kind=ScalarOutcome.MINIMIZE),
        ScalarOutcome(
            "max_distr_time",
            kind=ScalarOutcome.MINIMIZE,
        )
    ]

    if [o.name for o in EMA_model_dict[trace_label].outcomes] != oois:
        print("Watch out! Oois and model outcome names are not similar!")

    nondominated_BE_policies_EMA = []
    for name, pol in zip(nondom_i, nondominated_BE_policies):
        nondominated_BE_policies_EMA.append(Policy("{}".format(name), **pol))

    time1 = time.time()
    all_scen_results_nondom_pols = perform_experiments(
        EMA_model_dict[trace_label], n_scenarios, nondominated_BE_policies_EMA)
    time2 = time.time()
    print("Total time:{}".format(time2 - time1), "\n",
          "time per run = {}".format(
              (time2 - time1) /
              (n_scenarios * len(nondominated_BE_policies_EMA))))

    ema.save_results(all_scen_results_nondom_pols,
                     "results/mpmordm/{}.tar.gz".format(trace_label))
    experiments, outcomes = all_scen_results_nondom_pols

    sigN_results = []

    for policy in nondominated_BE_policies_EMA:
        #filter the outcome values corresponding to each policy using a logical index
        logical_index = experiments['policy'] == policy.name
        # If ever something goes wrong here, probably add: int() to policy.name
        pol_outcomes = {
            key: outcomes[key][logical_index]
            for key in outcomes.keys()
        }
        temp_result = {}
        for ooi in EMA_model_dict[trace_label].outcomes:
            temp_result[ooi.name] = calculate_signalNoise(
                pol_outcomes[ooi.name], ooi.kind)
        sigN_results.append(temp_result)

    df_sigN = pd.DataFrame(
        sigN_results, index=[pol.name for pol in nondominated_BE_policies_EMA])

    reg_results = {}
    # we need to iterate over the outcomes differently now,
    #because the maximum egret calculation requires the outcome values for all policies
    for ooi in EMA_model_dict[trace_label].outcomes:
        data = []
        for policy in nondominated_BE_policies_EMA:
            logical_index = experiments['policy'] == policy.name
            # If ever something goes wrong here, add int() to policy.name
            data.append(outcomes[ooi.name][logical_index])
        reg_results[ooi.name] = calculate_maxregret(data, ooi.kind)
    df_regret = pd.DataFrame.from_dict(
        reg_results,
        orient='columns',
    )
    df_regret.index = [pol.name for pol in nondominated_BE_policies_EMA]

    all_robustness_scores = pd.merge(
        df_regret,
        df_sigN,
        left_index=True,
        right_index=True,
        suffixes=("_regret", "_sigN"))

    robust_nondominated_policies = pareto.eps_sort(
        [list(all_robustness_scores.itertuples(index=True))],
        [i + 1 for i in range(all_robustness_scores.shape[1])],
        [1e-9] * all_robustness_scores.shape[1])

    if len(robust_nondominated_policies) == 0:
        print("something went wrong")
    elif len(robust_nondominated_policies) == 1:
        print("no branching, only a single nondominated robust location")
        temp_cols = ["pol_i"] + list(all_robustness_scores.columns)

        df_rob_nondom_pols = pd.DataFrame(
            robust_nondominated_policies, columns=temp_cols).set_index("pol_i")
    #     print(robust_nondominated_policies)
    else:
        temp_cols = ["pol_i"] + list(all_robustness_scores.columns)
        df_rob_nondom_pols = pd.DataFrame(
            robust_nondominated_policies, columns=temp_cols).set_index("pol_i")

    df_rob_nondom_pols.to_csv(
        "results/mpmordm/nondom_rob_pols{}.csv".format(trace_label))

    rob_nondom_i = [int(i) for i in df_rob_nondom_pols.index]
    #     return df_rob_nondom_pols
    
    return np.array(optional_policies)[rob_nondom_i]

## Testing single cycle

TODO: save csv nondom pols + csv rob nondom pols
possible: clear op variable space with del

In [25]:
test = MORDM(
    current=start_situation,n_scenarios=10, trace_label=start_label)

[MainProcess/INFO] performing 10 scenarios * 4 policies * 1 model(s) = 40 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 4 cases completed
[MainProcess/INFO] 8 cases completed
[MainProcess/INFO] 12 cases completed
[MainProcess/INFO] 16 cases completed
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 24 cases completed
[MainProcess/INFO] 28 cases completed
[MainProcess/INFO] 32 cases completed
[MainProcess/INFO] 36 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] results saved successfully to C:\Users\timro\Dropbox\MSc EPA\0 Master Thesis\Python Files\Thesis\Facility Location Model\results\mpmordm\p0n0.tar.gz


Total time:1.6586172580718994 
 time per run = 0.041465431451797485


# Full cycle

In [9]:
# # for mpmordm
# start_situation = {key:0 for key in FLs.Name}
# global_policy_tree[start_label] = start_situation

In [36]:
global_dfs_var_unc_data.keys()

dict_keys(['p0n0'])

Now first: label management

In [48]:
def change_decision_based_uncertainty(current_uncertain_data):
    return current_uncertain_data

In [49]:
periods = 2 # 1 location per period -> locations = periods

global_node_counter = 0

start_situation = {key:0 for key in FLs.Name}
global_policy_tree = {start_label : start_situation}
global_label_tree = {
    "p{}".format(p, global_node_counter): []
    if p != 0 else ["{}".format(start_label)]
    for p in range(periods+1)
}

EMA_model_dict = {}  # XX somewhere else: global
print("starting label:", start_label)


for p in range(periods):
    for node_i, node in enumerate(list(global_label_tree["p{}".format(p)])):
#         print("|", p, node)
        if p < periods:
            print("Do mordm, in period:", p,"label:", node,"policy:",global_policy_tree[node])
#             new_pols = ["a", "b", "c"]
            new_pols = MORDM(current=global_policy_tree[node],n_scenarios=10, trace_label=node)
            for new_pol in new_pols:
                # Create new label
                global_node_counter += 1
                new_label = node + "p{}n{}".format(p + 1, global_node_counter)
                print("result:|||",new_label)
                # Create new branch
                global_label_tree["p{}".format(p + 1)].append(new_label)
                # add new policy
                global_policy_tree[new_label] = new_pol
                # change information based on policy
                #TODO not implemented yet
#                 global_dfs_var_unc_data[new_label] = global_dfs_var_unc_data[start_label]
                global_dfs_var_unc_data[new_label] = change_decision_based_uncertainty(global_dfs_var_unc_data[node])

                
                
                
        else:
            print(p,
                "if you see this, sth went wrong."
            )
#     print("__end_period:",p)

starting label: p0n0
Do mordm, in period: 0 label: p0n0 policy: {'FLO0': 0, 'FLO1': 0, 'FLO2': 0, 'FLO3': 0, 'FLO4': 0, 'FLO5': 0, 'FLO6': 0, 'FLO7': 0, 'FLO8': 0, 'FLO9': 0, 'FLO10': 0, 'FLO11': 0, 'FLO12': 0, 'FLO13': 0, 'FLO14': 0, 'FLO15': 0, 'FLO16': 0, 'FLO17': 0, 'FLO18': 0, 'FLO19': 0}


[MainProcess/INFO] performing 10 scenarios * 4 policies * 1 model(s) = 40 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 4 cases completed
[MainProcess/INFO] 8 cases completed
[MainProcess/INFO] 12 cases completed
[MainProcess/INFO] 16 cases completed
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 24 cases completed
[MainProcess/INFO] 28 cases completed
[MainProcess/INFO] 32 cases completed
[MainProcess/INFO] 36 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] results saved successfully to C:\Users\timro\Dropbox\MSc EPA\0 Master Thesis\Python Files\Thesis\Facility Location Model\results\mpmordm\p0n0.tar.gz


Total time:1.887394666671753 
 time per run = 0.04718486666679382
result:||| p0n0p1n1
result:||| p0n0p1n2
Do mordm, in period: 1 label: p0n0p1n1 policy: {'FLO0': 0.0, 'FLO1': 0.0, 'FLO2': 0.0, 'FLO3': 1.0, 'FLO4': 0.0, 'FLO5': 0.0, 'FLO6': 0.0, 'FLO7': 0.0, 'FLO8': 0.0, 'FLO9': 0.0, 'FLO10': 0.0, 'FLO11': 0.0, 'FLO12': 0.0, 'FLO13': 0.0, 'FLO14': 0.0, 'FLO15': 0.0, 'FLO16': 0.0, 'FLO17': 0.0, 'FLO18': 0.0, 'FLO19': 0.0}


[MainProcess/INFO] performing 10 scenarios * 4 policies * 1 model(s) = 40 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 4 cases completed
[MainProcess/INFO] 8 cases completed
[MainProcess/INFO] 12 cases completed
[MainProcess/INFO] 16 cases completed
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 24 cases completed
[MainProcess/INFO] 28 cases completed
[MainProcess/INFO] 32 cases completed
[MainProcess/INFO] 36 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] results saved successfully to C:\Users\timro\Dropbox\MSc EPA\0 Master Thesis\Python Files\Thesis\Facility Location Model\results\mpmordm\p0n0p1n1.tar.gz


Total time:1.595156192779541 
 time per run = 0.039878904819488525
result:||| p0n0p1n1p2n3
result:||| p0n0p1n1p2n4
result:||| p0n0p1n1p2n5
result:||| p0n0p1n1p2n6
Do mordm, in period: 1 label: p0n0p1n2 policy: {'FLO0': 0.0, 'FLO1': 0.0, 'FLO2': 0.0, 'FLO3': 0.0, 'FLO4': 0.0, 'FLO5': 0.0, 'FLO6': 0.0, 'FLO7': 0.0, 'FLO8': 0.0, 'FLO9': 0.0, 'FLO10': 0.0, 'FLO11': 0.0, 'FLO12': 0.0, 'FLO13': 1.0, 'FLO14': 0.0, 'FLO15': 0.0, 'FLO16': 0.0, 'FLO17': 0.0, 'FLO18': 0.0, 'FLO19': 0.0}


[MainProcess/INFO] performing 10 scenarios * 5 policies * 1 model(s) = 50 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 5 cases completed
[MainProcess/INFO] 10 cases completed
[MainProcess/INFO] 15 cases completed
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 25 cases completed
[MainProcess/INFO] 30 cases completed
[MainProcess/INFO] 35 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] 45 cases completed
[MainProcess/INFO] 50 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] results saved successfully to C:\Users\timro\Dropbox\MSc EPA\0 Master Thesis\Python Files\Thesis\Facility Location Model\results\mpmordm\p0n0p1n2.tar.gz


Total time:2.0058581829071045 
 time per run = 0.04011716365814209
result:||| p0n0p1n2p2n7
result:||| p0n0p1n2p2n8
result:||| p0n0p1n2p2n9
result:||| p0n0p1n2p2n10


In [47]:
global_label_tree

{'p0': ['p0n0'],
 'p1': ['p0n0p1n1', 'p0n0p1n2'],
 'p2': ['p0n0p1n1p2n3',
  'p0n0p1n1p2n4',
  'p0n0p1n1p2n5',
  'p0n0p1n1p2n6',
  'p0n0p1n2p2n7',
  'p0n0p1n2p2n8',
  'p0n0p1n2p2n9',
  'p0n0p1n2p2n10']}

In [50]:
global_label_tree

{'p0': ['p0n0'],
 'p1': ['p0n0p1n1', 'p0n0p1n2'],
 'p2': ['p0n0p1n1p2n3',
  'p0n0p1n1p2n4',
  'p0n0p1n1p2n5',
  'p0n0p1n1p2n6',
  'p0n0p1n2p2n7',
  'p0n0p1n2p2n8',
  'p0n0p1n2p2n9',
  'p0n0p1n2p2n10']}

In [59]:
for key in global_policy_tree.keys():
    poldict = global_policy_tree[key]
    print(key,[key2 for key2 in poldict if poldict[key2]==1])

p0n0 []
p0n0p1n1 ['FLO3']
p0n0p1n2 ['FLO13']
p0n0p1n1p2n3 ['FLO0', 'FLO3']
p0n0p1n1p2n4 ['FLO3', 'FLO11']
p0n0p1n1p2n5 ['FLO3', 'FLO13']
p0n0p1n1p2n6 ['FLO3', 'FLO19']
p0n0p1n2p2n7 ['FLO0', 'FLO13']
p0n0p1n2p2n8 ['FLO3', 'FLO13']
p0n0p1n2p2n9 ['FLO11', 'FLO13']
p0n0p1n2p2n10 ['FLO13', 'FLO19']


In [77]:
for key in global_policy_tree.keys():
    if "n2" in key:
        poldict = global_policy_tree[key]
        print(key,[key2 for key2 in poldict if poldict[key2]==1])

p0n0p1n2 ['FLO13']
p0n0p1n2p2n7 ['FLO0', 'FLO13']
p0n0p1n2p2n8 ['FLO3', 'FLO13']
p0n0p1n2p2n9 ['FLO11', 'FLO13']
p0n0p1n2p2n10 ['FLO13', 'FLO19']


In [ ]:
EMA_model_dict = {}

In [59]:
def MORDM_cycle(current, last, trace_label, fixed_data, var_data, max_depth = 2, additional_info=None):
    """
    MORDM_cycle with micromodel. Micromodel changes factors/information based on the last decision
    
    """
    if trace_label != max_depth:
        raise NotImplementedError ("stop if max depth is reached")
    # micromodel here
    #Based on current / last
    
    
    MORDM(current, trace_label, fixed_data, var_data, max_depth = max_depth)